# PRAD model extension combined list of oncogenes, TGFBeta genes and cell cycle genes

Here we will experiment with adding new features based on DE genes. The DE-genes have been filtered and ranked based on feature elimination utilising RSFs.

In [1]:
library(tidyverse)
library(survival)
library(survminer)
library(glmnet)
library(WriteXLS)
library(ggfortify)
library(circlize)
library(ComplexHeatmap)
library(parallel)
library(broom)
library(survcomp)
library(survivalROC)
library(gtsummary)
source("../getTCGAData.R")
source("../preprocessTCGAData.R")
source("../KM_analysis.R")
source("../Heatmaps.R")
source("../enet.R")
library("readxl")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.2.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: ggpubr


Attaching package: ‘survminer’


The following object is masked from ‘package:survival’:

    myeloma


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-8

circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.

## Functions 

In [2]:
#
#
#
# Helper function for fixing variable names 
fixVarNames = function(x){
    print(x[1])
    if (str_detect(x, "Gender.clin")) {
        return("Gender")
    } else if (str_detect(x, "Tumor.stage.clin")){
        return("Tumor.stage")
    } else if (str_detect(x,".cn")){
        return(str_extract(x, "\\w+.cn"))
    } else if (str_detect(x, "Gleason.group.clin")){ 
        return("Gleason.group.clin")
    } else {
        return(x)
    }
}


# 
# Function fits a cox regression model
# 
fitCoxModel = function(data, end.point, features){
    
    # Expand to variable name
    end_point_time = paste0(end.point, ".time.clin")
    end_point_event = paste0(end.point, ".clin")

    # Generate a survival formula object 
    survExpression = paste0("Surv(", end_point_time, ", " , end_point_event, ")")
    f <- as.formula(paste(survExpression, paste(features, collapse = " + "), sep = " ~ "))
    
    model.fit = coxph(f, data = data)
    return(model.fit)
}

## Load processed data

In [3]:
tcga.dataset.merged = readRDS("/lustre/projects/landstrom_core/data/rdata_revised/manuscript_work//PRAD//tcga.dataset_merged.rds")

Convert - to _

In [4]:
colnames(tcga.dataset.merged$PFI$train) = str_replace(colnames(tcga.dataset.merged$PFI$train), "-", "_")
colnames(tcga.dataset.merged$PFI$validation) =  str_replace(colnames(tcga.dataset.merged$PFI$validation), "-", "_")

## Load original model and extracting features

In [5]:
# Loading original features 
original.model.features = read_excel("/lustre/projects/landstrom_core/results/prognostic_model_development_revised/models_by_cancer_type/PRAD/Penalized_Cox_risk_prediction/customer_features/With_clinical_features/PFI/Active_covariates_in_lambda.min_model.xlsx")

New names:
• `` -> `...1`


In [6]:
original.model.features

...1,coef,HR
<chr>,<dbl>,<dbl>
Age.clin,2.777648e-06,1.0000028
Gleason.group.clinGleason_group_2,9.138624e-01,2.4939365
KIF23.exp,3.846910e-01,1.4691603
PTEN.exp,-4.741249e-01,0.6224295
VPS4B.exp,-2.241524e-01,0.7991933
AURKA.exp,-4.645377e-01,0.6284256
AURKB.exp,4.394660e-01,1.5518783
TP53.cn-1,7.668698e-02,1.0797041


In [7]:
# Feature names to be selected 
selected.features = map_chr(unlist(original.model.features[,1]), fixVarNames) %>% as.vector()

[1] "Age.clin"
[1] "Gleason.group.clinGleason_group_2"
[1] "KIF23.exp"
[1] "PTEN.exp"
[1] "VPS4B.exp"
[1] "AURKA.exp"
[1] "AURKB.exp"
[1] "TP53.cn-1"


## Refitting the original model using standard cox-regression

In [8]:
# Construct the clinical end points 
end_point_event = paste0("PFI", ".clin")
end_point_time = paste0("PFI", ".time.clin")
    
selected.columns = c(end_point_event, end_point_time, selected.features)

In [9]:
# Input data for cox-model 
input.training = tcga.dataset.merged[["PFI"]]$train %>% dplyr::select(all_of(selected.columns))

# Fit the cox-model 
pcox.ref.fit.pfi.original = fitCoxModel(input.training, "PFI", selected.features)

# Predict the risk scores for validation data 
rel.risk = predict(object = pcox.ref.fit.pfi.original, 
                   newdata = tcga.dataset.merged$PFI$validation[,selected.features], 
                   type = "risk")

#
# Evaluate 
#

# Stratify validation data into two groups based on the fitted relative risk
y.data <- tcga.dataset.merged$PFI$validation[paste0("PFI", c(".clin",".time.clin"))]
colnames(y.data) = c("status","time")

# TEST new function for calculating the C-index
cindex.ref.valid = concordance.index(rel.risk, 
                                        y.data$time, 
                                        y.data$status,
                                        na.rm = TRUE)

# Plot KM and extract the p-value  
KM.valid.ref.by.risk = plotKMbyRelativeRisk(data = y.data, 
                                                     rel.risk = rel.risk)

In [10]:
# Modify the labels 

colnames(tcga.dataset.merged[["PFI"]]$train) = str_replace(colnames(tcga.dataset.merged[["PFI"]]$train), "-", "_")
colnames(tcga.dataset.merged[["PFI"]]$validation) = str_replace(colnames(tcga.dataset.merged[["PFI"]]$validation), "-", "_")

## Load the feature elimination results

In [11]:
feature.elim.res = readRDS("/lustre/projects/landstrom_core/data/rdata/manuscript_work/revisions/combined_list/PRAD/Feature_elimination_res_PFI_oncogenes.rds")

In [22]:
feature.elim.res$Errors

ntree,mtry,nodesize,num_variable,error
<dbl>,<dbl>,<dbl>,<int>,<dbl>
500,2,9,538,0.2746049
50,2,8,430,0.2726204
100,1,3,344,0.2932745
200,1,9,275,0.2712238
200,1,3,220,0.2543183
1000,1,5,176,0.2588019
100,2,2,140,0.2345829
1000,1,6,112,0.2338846
500,1,6,89,0.2321940


In [12]:
length(feature.elim.res$Features)

[1] 17

In [14]:
genes.to.add = feature.elim.res$Optimal.var$Variable
genes.to.add = str_replace(genes.to.add, "-", "_")

In [15]:
# Store results 
step.wise.results = matrix(nrow = 0, ncol = 2) %>% as.data.frame() 
colnames(step.wise.results) = c("KM.pvalue","C.index")

In [16]:
step.wise.results = as.data.frame(lapply(step.wise.results, as.numeric))

In [17]:
selected.columns.stepwise = selected.columns

In [19]:
for (i in 1:length(genes.to.add)){
    new.feature = genes.to.add[i]
    
    # Test adding one gene
    selected.columns.stepwise = c(selected.columns.stepwise, new.feature)
    
    # Input data for cox-model 
    input.training = tcga.dataset.merged[["PFI"]]$train[,colnames(tcga.dataset.merged[["PFI"]]$train) %in% selected.columns.stepwise]

    # The features 
    selected.features.stepwise = selected.columns.stepwise[-c(1,2)]
    
    # Fit the cox-model 
    pcox.ref.fit.pfi = fitCoxModel(input.training, "PFI", selected.features.stepwise)

    # Predict the risk scores for validation data 
    rel.risk = predict(object = pcox.ref.fit.pfi , 
                   newdata = tcga.dataset.merged$PFI$validation[,selected.features.stepwise], 
                   type = "risk")

    #
    # Evaluate 
    #

    # Stratify validation data into two groups based on the fitted relative risk
    y.data <- tcga.dataset.merged$PFI$validation[paste0("PFI", c(".clin",".time.clin"))]
    colnames(y.data) = c("status","time")

    # TEST new function for calculating the C-index
    cindex.ref.valid = concordance.index(rel.risk, 
                                        y.data$time, 
                                        y.data$status,
                                        na.rm = TRUE)

    # Plot KM and extract the p-value  
    KM.valid.ref.by.risk = plotKMbyRelativeRisk(data = y.data, 
                                                     rel.risk = rel.risk)
    
    
    res = data.frame(KM.pvalue = KM.valid.ref.by.risk$table$Pvalue.pval,
                     C.index = cindex.ref.valid$c.index,
                     "C-index CI" = paste0("(", round(cindex.ref.valid$lower, 4), " - ",  
                                                round(cindex.ref.valid$upper, 4), ")")
                     )
    
    step.wise.results = rbind(step.wise.results, res)
    
    #print(cindex.ref.valid$c.index)
    #print(KM.valid.ref.by.risk$table)
    
}

Find the minimal p-value and the corresponding C-index

In [21]:
step.wise.results

KM.pvalue,C.index,C.index.CI
<dbl>,<dbl>,<chr>
0.03276572,0.6086270,(0.3708 - 0.8041)
0.04265400,0.6170437,(0.3782 - 0.8102)
0.29508967,0.5833772,(0.3487 - 0.7855)
0.15443495,0.5854813,(0.3505 - 0.7871)
0.29732351,0.5854813,(0.3505 - 0.7871)
0.27267591,0.5833772,(0.3487 - 0.7855)
0.16334542,0.5839032,(0.3492 - 0.7859)
0.25271704,0.5917938,(0.356 - 0.7917)
0.63538282,0.5796949,(0.3455 - 0.7828)


In [20]:
which(step.wise.results$KM.pvalue == min(step.wise.results$KM.pvalue))

[1] 1

In [25]:
step.wise.results[1,]

,KM.pvalue,C.index,C.index.CI
,<dbl>,<dbl>,<chr>
1,0.03276572,0.608627,(0.3708 - 0.8041)


In [26]:
genes.to.add[1]

[1] "ALOX12B.exp"

In [35]:
write.csv(feature.elim.res$Optimal.var, "Feature_elimination_combined_temp.csv")